<a href="https://colab.research.google.com/github/TayaMedvedeva/STA_gain_adjustment_Python/blob/main/step_test_based_procedure_for_super_twisting_algorithm_gains_adjustment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# First step - Identification

The file with step-test can be uploaded in CSV format:

*   the first colunm - time
*   the second colunm - output
*   dt - sampling time
*   Новый пункт






In [ ]:
# Add your data
rec = pd.read_csv('drive/MyDrive/Colab Notebooks/sta_autom/Andres_experiment/U5.csv') # path to the file
dt = 0.001 # sampling time
input = 5 # input value
Delta = 1 # Lipzich constant
###---###

In [ ]:
# Identification
rec = np.array(rec)
t = rec[:, 0]
y = rec[:, 2]

dy = np.gradient(y, dt)
y_ss = np.mean(y[len(y) - 500 : len(y)])
parameter_k = y_ss / input

dy_max = dy[0]
t_inf= 0
i_inf = 0
t_start = 0
for i in range(len(dy)):
    if dy_max < dy[i]:
        dy_max = dy[i]
        t_inf = t[i]
        i_inf = i
    if y[i] == 0:
        t_start = t[i]

tg = dy_max
y_inf = y[i_inf]
parameter_tau = -(y_inf - tg * t_inf) / tg
parameter_v = (y_ss - (y_inf - tg * t_inf)) / tg - parameter_tau


print('k = ', parameter_k)
print('tau = ', parameter_tau - t_start)
print('v = ', parameter_v)


k =  1.7544476419073554
tau =  0.029660448016120444
v =  0.654263697797203


# Second step - STA gain adjustment

In [ ]:
# Add your identified parameters
k =  1.752618504602958
tau =  0.038253814491698845
v =  0.8734923360143185
###---###

k2 = 1.1 * Delta
pi = np.pi
alpha = 1.748

In [ ]:
def func_w_cardan(k11, k22, v, tau):
    a1 = -alpha**2*k *tau**4*v
    b1 = 4*alpha**2*k*tau**3
    b2 = - 1.0*pi*tau**4 - 8.0*pi*tau**3*v - 16.0*pi*tau**2*v**2
    c1 = 16*alpha**2*k*tau + 16*alpha**2*k*v
    c2 = 8.0*pi*tau**2 + 32.0*pi*tau*v
    d2 = - 16*pi

    a = a1 * k11**2
    b = b1 * k11**2 + b2 * k22
    c = c1 * k11**2 + c2 * k22
    d = d2 * k22

    Q = (3*a*c - b**2)/(9*a**2)
    R = (9*a*b*c - 27*a**2*d - 2*b**3) / (54*a**3)

    S = (R + (Q**3 + R**2) ** (1/2)) ** (1/3)
    T = (R - (Q**3 + R**2) ** (1/2)) ** (1/3)

    xi_1_sol = S + T - b / (3*a)
    xi_2_sol = -(S + T)/2 - b / (3*a) + 1j*np.sqrt(3)/2*(S - T)
    xi_3_sol = -(S + T)/2 - b / (3*a) - 1j*np.sqrt(3)/2*(S - T)


    w_1_sol = np.real((xi_1_sol) ** (1/2))
    w_2_sol = np.real((xi_2_sol) ** (1/2))
    w_3_sol = np.real((xi_3_sol) ** (1/2))

    w_sol = np.sort(np.array([w_1_sol, w_2_sol, w_3_sol]))

    return [w_sol[0], w_sol[1], w_sol[2]]


def get_A(w_arr, k11, k22, v, tau):
    A_arr = []
    for i in range(3):
        w = w_arr[i]
        A = 4.0*alpha**2*k**2*k11**2*(0.25*tau**2*w**2 + 1)**2/(pi**2*(0.25*tau**2*w**2 + tau*v*w**2 - 1)**2)
        A_arr.append(A)
    return A_arr


def find_k1(k, tau, v):
    n = 500
    k1_start = 0.01
    k1_end = 5
    step = (k1_end - k1_start) / n
    k1_arr = [k1_start + step*i for i in range(n)]

    A_min = np.inf
    P_min = np.inf
    w_sol, A_sol = [], []
    for i in range(n):
        k1 = k1_arr[i]
        w0 = func_w_cardan(k1, k2, v, tau)
        w1_0, w2_0, w3_0 = w0[0], w0[1], w0[2]

        w = [w1_0, w2_0, w3_0]
        A = get_A(w, k1, k2, v, tau)
        w_sol.append(w)
        A_sol.append(A)

        if A[2] < A_min:
            k1_Ac = k1
            A_min = A[2]

    w_sol = np.array(w_sol)
    A_sol = np.array(A_sol)
    return k1_Ac


k1_Ac = find_k1(k, tau, v)
print('k1_Ac_num = ', k1_Ac)

k1_Ac_num =  1.03794
